In [1]:
%cd ../..
%run cryptolytic/notebooks/init.ipynb
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import cryptolytic.util as util
import cryptolytic.start as start
import cryptolytic.viz.plot as plot
import cryptolytic.data.sql as sql
import cryptolytic.data as d
from cryptolytic.util import *
import cryptolytic.data.historical as h
import cryptolytic.model as m

from matplotlib.pylab import rcParams
from IPython.core.display import HTML
from pandas.plotting import register_matplotlib_converters # to stop a warning message


ohclv = ['open', 'high', 'close', 'low', 'volume']
plt.style.use('ggplot')
rcParams['figure.figsize'] = 20,7
start.init()
register_matplotlib_converters()


# Make math readable
HTML("""
<style>
.MathJax {
    font-size: 2rem;
}
</style>""")

/home/me/Documents/Git/Lambda-School-Labs/cryptolytic-ds
Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

### LSTM with Memory Between Batches

>We can gain finer control over when the internal state of the LSTM network is cleared in Keras by making the LSTM layer “stateful”. This means that it can build state over the entire training sequence and even maintain that state if needed to make predictions.

>It requires that the training data not be shuffled when fitting the network. It also requires explicit resetting of the network state after each exposure to the training data (epoch) by calls to model.reset_states(). This means that we must create our own outer loop of epochs and within each epoch call model.fit() and model.reset_states(). For example:

```{python}
for i in range(100):
	model.fit(trainX, trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()
```

>Finally, when the LSTM layer is constructed, the stateful parameter must be set True and instead of specifying the input dimensions, we must hard code the number of samples in a batch, number of time steps in a sample and number of features in a time step by setting the batch_input_shape parameter. For example:
```
model.add(LSTM(4, batch_input_shape=(batch_size, time_steps, features), stateful=True))
```

In [2]:
# TRAIN_SPLIT = 5000
history_size = 1000
input_len = 8500
train_size = int(5000*0.8)
lahead = 10
step = 2
rolling_size = 6
period = 300
batch_size = 100
to_drop = max(rolling_size - 1, lahead - 1)
input_len = input_len + to_drop
df = d.get_df ({'start':'06-01-2019', 'period':period, 'trading_pair':'eth_btc', 'exchange_id':'bitfinex'},
               n=input_len)

/home/me/Documents/Git/Lambda-School-Labs/cryptolytic-ds/cryptolytic/data/__init__.py:54: FutureWarning:

how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)



In [3]:
c = df[['close', 'volume', 'diff', 'arb_signal', 'timestamp']]
a_df = c.rolling(rolling_size).mean().bfill().rename(columns=lambda x: x+'_mean')
b_df = c.rolling(rolling_size).std().bfill().rename(columns=lambda x: x+'_std')
c_df = c.rolling(rolling_size).skew().bfill().rename(columns=lambda x: x+'_skew')
d_df = c.rolling(rolling_size).kurt().bfill().rename(columns=lambda x: x+'_kurt')
df = pd.concat([c, a_df, b_df, c_df, d_df], axis=1).dropna(axis=1)
# df_sub = df.drop(['timestamp', 'period', 'open', 'high', 'low', 'api', 'exchange', 'trading_pair'], axis=1)
df.head()

,close,volume,diff,arb_signal,timestamp,close_mean,volume_mean,diff_mean,arb_signal_mean,timestamp_mean,...,close_skew,volume_skew,diff_skew,arb_signal_skew,timestamp_skew,close_kurt,volume_kurt,diff_kurt,arb_signal_kurt,timestamp_kurt
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-06 06:00:00,0.039829,197.424839,0.000001,-0.047700,1546754400,0.039841,127.217718,0.000033,-0.059587,1.546755e+09,...,-0.269907,0.780103,-0.015923,0.478313,-11031.203129,-0.417309,-0.249835,-1.893313,-0.417291,2.032476e+11
2019-01-06 06:05:00,0.039867,320.041091,0.000063,-0.087709,1546754700,0.039841,127.217718,0.000033,-0.059587,1.546755e+09,...,-0.269907,0.780103,-0.015923,0.478313,-11031.203129,-0.417309,-0.249835,-1.893313,-0.417291,2.032476e+11
2019-01-06 06:10:00,0.039870,70.079043,0.000001,-0.090318,1546755000,0.039841,127.217718,0.000033,-0.059587,1.546755e+09,...,-0.269907,0.780103,-0.015923,0.478313,-11031.203129,-0.417309,-0.249835,-1.893313,-0.417291,2.032476e+11
2019-01-06 06:15:00,0.039840,18.358491,0.000030,-0.048922,1546755300,0.039841,127.217718,0.000033,-0.059587,1.546755e+09,...,-0.269907,0.780103,-0.015923,0.478313,-11031.203129,-0.417309,-0.249835,-1.893313,-0.417291,2.032476e+11
2019-01-06 06:20:00,0.039804,9.974051,0.000037,-0.067172,1546755600,0.039841,127.217718,0.000033,-0.059587,1.546755e+09,...,-0.269907,0.780103,-0.015923,0.478313,-11031.203129,-0.417309,-0.249835,-1.893313,-0.417291,2.032476e+11


In [4]:
def normalize(df):
    df = df.copy()
    df = df._get_numeric_data()
    # normalize each point by dividing by the inital point and subtracting 1
    for col in df.columns: 
        df[col] = (df[col] / (df[col][0] + 1e-7)) - 1
    return df
    
def denormalize(values, df, col=None):
    values = values.copy()
    if np.ndim(values) == 1:
        return (1 + values) * df[col][0]
    else:
        for i in range(values.shape[1]):
            if isinstance(values, pd.DataFrame): 
                values.iloc[:, i] = (1 + values.iloc[:, i]) * df.iloc[:, i][0]
            else:
                values[i] = (1 + values[:, i]) * df.iloc[:, i][0]
        return values

In [5]:
df = df.sort_index()
# dataset = normalize_df(df._get_numeric_data()).values
dataset = normalize(df._get_numeric_data())
target = dataset.columns.get_loc('close') 
dataset = dataset.values
y = dataset[:, target]

# x_train, y_train = multivariate_data(dataset, y, 0, TRAIN_SPLIT, past_history, lahead, STEP, single_step=False)
# x_val, y_val = multivariate_data(dataset, y,TRAIN_SPLIT, None, past_history,lahead, STEP,single_step=False)

In [6]:
def windowed(df, target, batch_size, history_size, step, lahead=1, ratio=0.8):
    xs = []
    ys = []
    
    x = dataset
    y = dataset[:, target]

    start = history_size # 1000
    end = df.shape[0] - lahead # 4990
    # 4990 - 1000 = 3990
    for i in range(start, end):
        # grab rows from i, to i+history_size
        indices = range(i-history_size, i, step)
        xs.append(x[indices])
        ys.append(y[i:i+lahead])
        
    xs = np.array(xs)
    ys = np.array(ys)
    
    nrows = xs.shape[0]
    train_size = int(nrows * ratio)
    # make sure the sizes are multiples of the batch size (needed for stateful lstm)
    train_size -= train_size % batch_size
    val_size = nrows - train_size
    val_size -= val_size  % batch_size
    total_size = train_size + val_size
    xs = xs[:total_size]
    ys = ys[:total_size]
    
    return xs[:train_size], ys[:train_size], xs[train_size:], ys[train_size:]

In [7]:
x_train, y_train, x_val, y_val = windowed(dataset, target, batch_size, history_size, step, lahead)
mapl(lambda x: x.shape, [x_train, y_train, x_val, y_val])

[(5900, 500, 25), (5900, 10), (1500, 500, 25), (1500, 10)]

In [8]:
# there is a slight error between the normlized values and the original values
df.iloc[1000:1000+len(y_train[0])]['close_mean'].values - denormalize(y_train[0], df, 'close_mean')

array([ 4.08857092e-05,  2.00505330e-05,  7.21719965e-06, -5.44946701e-06,
       -1.99528456e-05, -1.89525385e-05,  1.20579045e-05, -3.61306223e-06,
        1.88958450e-05,  1.90649688e-05])

In [9]:
#def multivariate_data(dataset, target, start_index, end_index, history_size,
#                      target_size, step, single_step=False):
#    data = []
#    labels = []
#    
#    start_index = start_index + history_size
#    if end_index is None:
#        end_index = len(dataset) - target_size
#    
#    for i in range(start_index, end_index):
#        indices = range(i-history_size, i, step)
#        data.append(dataset[indices])
#        
#        if single_step:
#            labels.append(target[i+target_size])
#        else:
#            labels.append(target[i:i+target_size])
#    
#    return np.array(data), np.array(labels)

In [10]:
print ('Single window of past history : {}'.format(x_train[0].shape))
print ('\n Target temperature to predict : {}'.format(y_train[0].shape))

Single window of past history : (500, 25)

 Target temperature to predict : (10,)


In [11]:
print ('Single window of past history : {}'.format(x_train[0].shape))
print ('\n Target temperature to predict : {}'.format(y_train[0].shape))

Single window of past history : (500, 25)

 Target temperature to predict : (10,)


In [12]:
import tensorflow as tf
BUFFER_SIZE = 10_000
BATCH_SIZE = 250
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()

/home/me/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/me/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/me/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/me/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,))

In [13]:
import tensorflow as tf
import tensorflow.keras.layers as layers

def lstm_model(train, stateful):
    model = tf.keras.models.Sequential()
# use_bias is True
    # batch size is 240 for the dataset instead of 256 for some reason
    model.add(layers.LSTM(5, return_sequences=True, input_shape=(train.shape[-2:])))
    model.add(layers.LSTM(5, activation='relu'))
              
    model.add(layers.Dense(lahead)) # global variable remove
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00005), loss='mse')
    return model

In [28]:
import tensorflow.keras.models as models
def create_model(df, stateful):
    model = models.Sequential()
    model.add(layers.InputLayer(batch_size=batch_size, input_shape=x_train.shape[-2:]))
    model.add(layers.Dense(128, use_bias=False))
    model.add(layers.Dropout(0.05)) 
    model.add(layers.LayerNormalization())
    model.add(layers.PReLU())
 #   model.add(layers.LSTM(64, 
 #                  batch_size=batch_size,
 #                  input_shape=x_train.shape[-2:],
 #                  stateful=stateful,
 #                  return_sequences=True)
 #            )
    model.add(layers.Dropout(0.05))
    # Time Distributed
    # Applies a layer to every temporal slice of an input. 
    # model.add(TimeDistributed(Dense(16)))
    model.add(layers.LSTM(64, activation='relu', return_sequences=True))
    model.add(layers.LSTM(64, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.LayerNormalization())
    model.add(layers.Dropout(0.05))
    model.add(layers.PReLU())
    model.add(layers.Dense(126, activation='relu'))
    model.add(layers.Dense(lahead))
    #model.compile(loss='mse', optimizer='adam') 
    model.compile(loss='mae', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001))
    return model


In [33]:
model = create_model(df, stateful=True)
history = {'loss':[],
          'val_loss':[]}

In [29]:
def run_model():
    epochs = 10
    for i in range(epochs):
        print(f'Epoch {i}')
        # batch size higher than 1 causes to fail, not sure
        model.fit(x_train, y_train, 
                           batch_size=batch_size,
                           epochs=1,
                           verbose=1,
                           use_multiprocessing=True,
                           workers=4,
                           validation_data = (x_val, y_val), 
                           shuffle=False)
        history['loss'].append(model.history.history['loss'])
        history['val_loss'].append(model.history.history['val_loss'])
        model.reset_states()
        
    return model
        
model = run_model()

Epoch 0
Train on 5900 samples, validate on 1500 samples
5900/5900 [==============================] - 71s 12ms/sample - loss: 0.0873 - val_loss: 0.0117
Epoch 1
Train on 5900 samples, validate on 1500 samples
5900/5900 [==============================] - 68s 12ms/sample - loss: 0.1722 - val_loss: 0.0596
Epoch 2
Train on 5900 samples, validate on 1500 samples
5900/5900 [==============================] - 67s 11ms/sample - loss: 0.0726 - val_loss: 0.0163
Epoch 3
Train on 5900 samples, validate on 1500 samples
5900/5900 [==============================] - 68s 12ms/sample - loss: 0.1606 - val_loss: 0.0578
Epoch 4
Train on 5900 samples, validate on 1500 samples
5900/5900 [==============================] - 66s 11ms/sample - loss: 0.1228 - val_loss: 0.0248


In [ ]:
model.summary()

In [ ]:
# BUFFER_SIZE = 10_000
# BATCH_SIZE = 100
# train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
# 
# val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
# val_data = val_data.batch(BATCH_SIZE).repeat()
# 
# history = model.fit(train_data,
#                 steps_per_epoch=38,
#                 shuffle=False,     
#                 epochs=1,
#                 verbose=1,
#                 use_multiprocessing=True,
#                 workers=4,
#                 validation_data=val_data,
#                 validation_steps=5) 

In [31]:
def multi_step_plot(history, true_future, prediction):
    plt.figure(figsize=(12, 6))
    num_in = create_time_steps(len(history))
    num_out = len(true_future)
    
    plt.plot(num_in, np.array(history[:, 1]), label='History')
    plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
        label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
            label='Predicted Future')
    plt.legend(loc='upper left')
    plt.show()
    
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(len(loss))
    
    plt.figure()
    
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()
    
    plt.show()

    
plot_train_history(history, 'Multi Step Training and validation loss') 

In [32]:
history.history['loss']


[0.12277464928515887]

# Predictions on the train set

In [ ]:
preds = denormalize(model.predict(x_train)[:, 0], df, 'close_mean')

In [ ]:
rcParams['figure.figsize'] = 20,3
yo = 1000
plt.plot(np.arange(yo), d.denoise(df['close'].iloc[history_size:yo+history_size], 5), label='actual')
plt.plot(range(yo), d.denoise(preds[:yo], 5), label='predicted');
plt.plot(range(yo), df.close_mean.values[history_size:yo+history_size], label='Mean')
plt.legend();

In [ ]:
# plt.title('All predictions')
# plt.plot(range(2000), d.denoise(preds[:2000], 5));

# Predictions on validation set

In [ ]:
yo = 1000
val_actual = df['close_mean'].iloc[len(x_train):]
val_preds = denormalize(model.predict(x_val)[:, 0], df, 'close_mean')
plt.plot(np.arange(yo), d.denoise(val_actual, 5), label='actual')
plt.plot(range(yo), d.denoise(preds[:yo], 5), label='predicted');
# plt.plot(range(yo), df.close_mean.values[history_size:yo+history_size], label='Mean')
plt.legend();

In [ ]:
# val_preds = denormalize_results(model.predict(x_val))
# # val_actual = df.close.iloc[TRAIN_SPLIT:]
# val_actual = denormalize_results(y_val[:, 0])
# plt.plot(np.arange(2000), d.denoise(val_actual[past_history:2000+past_history], 20), label='actual')
# plt.plot(range(2000), d.denoise(val_preds[:, 0][:2000], 20), label='predicted');
# plt.legend();

In [ ]:
len(x_val)

In [ ]:
yo

In [ ]:
preds[0:5]

In [ ]:
history_size